In [1]:
from vertexai.generative_models import GenerationConfig, GenerativeModel, Content, GenerativeModel, Part  
from google.cloud import aiplatform  
from IPython.display import Markdown,display,HTML


In [6]:
question='how many superbowl Vikings has won?'
model = GenerativeModel("gemini-1.5-pro-002")
response = model.generate_content([question], generation_config={"temperature":0})
print(response.text)

The Minnesota Vikings have appeared in four Super Bowls but have not won any of them.  They lost Super Bowl IV (to the Kansas City Chiefs), Super Bowl VIII (to the Miami Dolphins), Super Bowl IX (to the Pittsburgh Steelers), and Super Bowl XI (to the Oakland Raiders).



In [7]:
# test connection first
model = GenerativeModel("gemini-1.5-pro-002")
response = model.generate_content(["""how many superbowls has Rams won"""], generation_config={"temperature":0})
print(response.text)

The Rams franchise has won a total of **four** Super Bowls:

* **Super Bowl XXXIV (2000)**:  St. Louis Rams defeated the Tennessee Titans.
* **Super Bowl LVI (2022)**: Los Angeles Rams defeated the Cincinnati Bengals.
* **Super Bowl XIV (1980)**: Los Angeles Rams lost to the Pittsburgh Steelers.
* **Super Bowl XXXVI (2002)**: St. Louis Rams lost to the New England Patriots.



In [8]:
model = GenerativeModel("gemini-1.5-pro-002")
response = model.generate_content(["""
how many superbowls has Rams won, and the score"""], generation_config={"temperature":0})
print(response.text)

The Los Angeles Rams have won **two** Super Bowls:

* **Super Bowl XXXIV (2000):** Rams 23, Tennessee Titans 16
* **Super Bowl LVI (2022):** Rams 23, Cincinnati Bengals 20



In [ ]:
model = GenerativeModel("gemini-1.5-pro-002")
response = model.generate_content(["""
how many superbowls has Rams won, and the score, 
and how many points they win by"""], generation_config={"temperature":0})
print(response.text)

In [3]:
model = GenerativeModel("gemini-1.5-pro-001")
response = model.generate_content(["""how many superbowls has Rams won"""], generation_config={"temperature":0})
print(response.text)

The Los Angeles Rams have won **2** Super Bowls. 

* **Super Bowl XXXIV (2000):** Defeated the Tennessee Titans 23-16
* **Super Bowl LVI (2022):** Defeated the Cincinnati Bengals 23-20 



<font color='red'> gemini 002 is incorrect </font>

In [7]:

# deliberately provide a incorrect answer to the model to test whether history is indeed being used, the rams didn't win the 2018 superbowls
model_answer='''
The Los Angeles Rams have won **3** Super Bowls. 

* **Super Bowl XXXIV (2000):** Defeated the Tennessee Titans 23-16
* **Super Bowl LVI (2018):** Defeated the Philadephia Eagles 10-7 
* **Super Bowl LVI (2022):** Defeated the Cincinnati Bengals 23-20 
'''
messages=[
    Content(role='user', parts=[Part.from_text('how many superbowls has Rams won')]),
    Content(role='model', parts=[Part.from_text(model_answer)]),
    Content(role='user', parts=[Part.from_text('who won the next year')])
   
]
model = GenerativeModel("gemini-1.5-pro-001")

response = model.generate_content(contents=messages, generation_config={"temperature":0}) 
response.text

"You're asking about the Super Bowl winner in the year following each Rams victory:\n\n* **After Super Bowl XXXIV (2000):** The Baltimore Ravens won Super Bowl XXXV in 2001.\n* **After Super Bowl LVI (2018):** The New England Patriots won Super Bowl LIII in 2019.\n* **After Super Bowl LVI (2022):** The Kansas City Chiefs won Super Bowl LVII in 2023. \n"

In [44]:
def execute(client, messages: list, generation_config={"temperature":0}) -> str:
    """
    Sends a request to the client's `generate_content` method to interact with the language model.

    Args:
        client (Gemini model): The gemini model
        messages (list[Content]): A list of Content as chat history
        generation_config: generation configuration
        
    Returns:
        str: The content of the model's response.
    """
    response = client.generate_content(contents=messages, generation_config=generation_config) 
    return response.text


def create_single_text_Content(role, text):
    '''
    Create a Content object for history, containing only one part, which is a text
    Args:
        role: can be either 'user' or 'model'
        text: the content
    Return: Content
    '''
    return Content(role=role, parts=[Part.from_text(text)])
    

def update_chat_history(history: list, msg: str, role: str):
    """
    Updates the chat history by appending the latest response.

    Args:
        history (list): The list representing the current chat history.
        msg (str): The message to append.
        role (str): The role type ('user' or 'model')
    """
    history.append(create_single_text_Content(text=msg, role=role))


class ChatHistory(list):
    def __init__(self, messages: list | None = None, total_length: int = -1):
        """
        Initialise the list with a fixed total length.

        Args:
            messages (list | None): A list of initial messages, each message is a Content
            total_length (int): The maximum number of messages the chat history can hold.
        """
        if messages is None:
            messages = []

        super().__init__(messages)
        self.total_length = total_length

    def append(self, msg):
        """Add a Content to the queue.
        Args:
            msg (Content): The message to be added to the queue
        """
        if len(self) == self.total_length:
            self.pop(0)
        super().append(msg)
        

In [9]:
import re
from dataclasses import dataclass


@dataclass
class TagContentResult:
    """
    A data class to represent the result of extracting tag content.

    Attributes:
        content (List[str]): A list of strings containing the content found between the specified tags.
        found (bool): A flag indicating whether any content was found for the given tag.
    """

    content: list[str]
    found: bool


def extract_tag_content(text: str, tag: str) -> TagContentResult:
    """
    Extracts all content enclosed by specified tags (e.g., <thought>, <response>, etc.).

    Parameters:
        text (str): The input string containing multiple potential tags.
        tag (str): The name of the tag to search for (e.g., 'thought', 'response').

    Returns:
        dict: A dictionary with the following keys:
            - 'content' (list): A list of strings containing the content found between the specified tags.
            - 'found' (bool): A flag indicating whether any content was found for the given tag.
    """
    # Build the regex pattern dynamically to find multiple occurrences of the tag
    tag_pattern = rf"<{tag}>(.*?)</{tag}>"

    # Use findall to capture all content between the specified tag
    matched_contents = re.findall(tag_pattern, text, re.DOTALL)

    # Return the dataclass instance with the result
    return TagContentResult(
        content=[content.strip() for content in matched_contents],
        found=bool(matched_contents),
    )

In [31]:
import importlib
from src import tool_functions
importlib.reload(tool_functions)
from src.tool_functions import Tool, get_fn_signature, validate_arguments, convert_to_tool


In [15]:
import json

def get_current_weather(location: str, unit: str):
    """
    Get the current weather in a given location
    
    Args:
        location (str): The city and state, e.g. Madrid, Barcelona
        unit (str): The unit. It can take two values; "Felsius", "Fahrenheit"
    """
    if location == "Charlotte":
        return json.dumps({"temperature": 25, "unit": unit})
    else:
        return json.dumps({"temperature": 58, "unit": unit})

In [16]:
get_current_weather(location="Charlotte", unit="Fahrenheit")


'{"temperature": 25, "unit": "Fahrenheit"}'

In [32]:
weather_tool = convert_to_tool(get_current_weather)


In [33]:
json.loads(weather_tool.fn_signature)

{'name': 'get_current_weather',
 'description': '\n    Get the current weather in a given location\n    \n    Args:\n        location (str): The city and state, e.g. Madrid, Barcelona\n        unit (str): The unit. It can take two values; "Felsius", "Fahrenheit"\n    ',
 'parameters': {'properties': {'location': {'type': 'str'},
   'unit': {'type': 'str'}}}}

In [121]:
SYSTEM_PROMPT= """
You are a function calling AI model. You are provided with function signatures within <tools></tools> XML tags.
You may call one or more functions to assist with the user query. Don't make assumptions about what values to plug
into functions. Pay special attention to the properties 'types'. You should use those types as in a Python dict.
For each function call return a json object with function name and arguments within <tool_call></tool_call>
XML tags as follows:

<tool_call>
{"name": <function-name>,"arguments": <args-dict>,  "id": <monotonically-increasing-id>}
</tool_call>

Here are the available tools:

<tools>
%s
</tools>

if none of the functions is related to the question, answer the question using your own knowledge.
if you are provided with observations, you may answer the question based on that observation
"""


class ToolAgent:
    
    def __init__(self, tools: Tool | list[Tool]) -> None:
        
        self.client = GenerativeModel("gemini-1.5-pro-001")
        self.tools = tools if isinstance(tools, list) else [tools]
        self.tools_dict = {tool.name: tool for tool in self.tools}
        

    def add_tool_signatures(self) -> str:
        """
        Collects the function signatures of all available tools.

        Returns:
            str: A concatenated string of all tool function signatures in JSON format.
        """
        
        return "".join([tool.fn_signature for tool in self.tools])

    def process_tool_calls(self, tool_calls_content: list) -> dict:
        """
        Processes each tool call, validates arguments, executes the tools, and collects results.

        Args:
            tool_calls_content (list): List of strings, each representing a tool call in JSON format.

        Returns:
            dict: A dictionary where the keys are tool call IDs and values are the results from the tools.
        """
        observations = {}
        for tool_call_str in tool_calls_content:
            tool_call = json.loads(tool_call_str)
            tool_name = tool_call["name"]
            tool = tools_dict[tool_name]

            print_in_color(text=f"\nUsing Tool: {tool_name}", color='green')

            validated_tool_call = validate_arguments(
                tool_call, json.loads(tool.fn_signature)
            )
            print_in_color(text=f"\nTool call dict: \n{validated_tool_call}", color='green')


            result = tool.run(**validated_tool_call["arguments"])

            print_in_color(text=f"\nTool result: \n{result}", color='green')


            observations[validated_tool_call["id"]] = result

        return observations
    
    def run(self, user_msg: str,):
        """
        Handles the full process of interacting with the language model and executing a tool based on user input.

        Args:
            user_msg (str): The user's message that prompts the tool agent to act.

        Returns:
            str: The final output after executing the tool and generating a response from the model.
        """
            
        chat_history = ChatHistory(
            [
                create_single_text_Content(
                    text=SYSTEM_PROMPT % self.add_tool_signatures(),
                    role="user",
                ),
                create_single_text_Content(
                    text=user_msg, 
                    role="user"
                ),
            ]
        )
        
        print("-"*80)
        
        response = execute(self.client, messages=chat_history)
        tool_calls = extract_tag_content(str(response), "tool_call")
        
                       
        if tool_calls.found:
            observations = self.process_tool_calls(tool_calls.content)
            update_chat_history(history=chat_history, msg=f"Observation: {observations}", role="user")
        
        
        print("final answer!!!!!!!!")
        
        return execute(self.client, messages=chat_history)


tool_agent = ToolAgent(tools=[weather_tool])
temp=tool_agent.run("What is the weather today in Charlotte?")
print(temp)

--------------------------------------------------------------------------------


{1: '{"temperature": 25, "unit": "Fahrenheit"}'}
final answer!!!!!!!!
The weather in Charlotte is 25 degrees Fahrenheit. 



In [73]:
tag_content=extract_tag_content(temp, "tool_call")
print(tag_content)

TagContentResult(content=['{"name": "get_current_weather", "arguments": {"location": "Charlotte", "unit": "Fahrenheit"}, "id": 1}'], found=True)


# BREAK

In [123]:
tag_content.content

['{"name": "get_current_weather", "arguments": {"location": "Charlotte", "unit": "Fahrenheit"}, "id": 1}']

In [90]:
def print_in_color(text, color):
    display(HTML(f'<span style="color: {color};">{text}</span>'))

In [100]:
tools=[weather_tool]
tools_dict = {tool.name: tool for tool in tools}
print(tools_dict)



def process_tool_calls( tool_calls_content: list) -> dict:
   
    """
    Processes each tool call, validates arguments, executes the tools, and collects results.

    Args:
        tool_calls_content (list): List of strings, each representing a tool call in JSON format.

    Returns:
        dict: A dictionary where the keys are tool call IDs and values are the results from the tools.
    """
    
    observations = {}
    for tool_call_str in tool_calls_content:
        tool_call = json.loads(tool_call_str)
        tool_name = tool_call["name"]
        tool = tools_dict[tool_name]

        print_in_color(text=f"\nUsing Tool: {tool_name}", color='green')

        validated_tool_call = validate_arguments(
            tool_call, json.loads(tool.fn_signature)
        )
        print_in_color(text=f"\nTool call dict: \n{validated_tool_call}", color='green')
        

        result = tool.run(**validated_tool_call["arguments"])
        
        print(validated_tool_call["arguments"])
        
        print_in_color(text=f"\nTool result: \n{result}", color='green')


        observations[validated_tool_call["id"]] = result
        print(observations)
        
    return observations

print(process_tool_calls(tool_calls_content=tag_content.content))

{'get_current_weather': <src.tool_functions.Tool object at 0x7fc172354340>}


{'location': 'Charlotte', 'unit': 'Fahrenheit'}


{1: '{"temperature": 25, "unit": "Fahrenheit"}'}
{1: '{"temperature": 25, "unit": "Fahrenheit"}'}


In [109]:
tool_call = json.loads(tag_content.content[0])
print(tool_call)
tool_signature=json.loads(weather_tool.fn_signature)
print(tool_signature)

{'name': 'get_current_weather', 'arguments': {'location': 'Charlotte', 'unit': 'Fahrenheit'}, 'id': 1}
{'name': 'get_current_weather', 'description': '\n    Get the current weather in a given location\n    \n    Args:\n        location (str): The city and state, e.g. Madrid, Barcelona\n        unit (str): The unit. It can take two values; "Felsius", "Fahrenheit"\n    ', 'parameters': {'properties': {'location': {'type': 'str'}, 'unit': {'type': 'str'}}}}


In [114]:

properties = tool_signature["parameters"]["properties"]
print(properties)
      
# TODO: This is overly simplified but enough for simple Tools.
type_mapping = {
        "int": int,
        "str": str,
        "bool": bool,
        "float": float,
}

for arg_name, arg_value in tool_call["arguments"].items():
    print(arg_name,arg_value)
    expected_type = properties[arg_name].get("type")
    print(type(expected_type))
    
    if not isinstance(arg_value, type_mapping[expected_type]):
        tool_call["arguments"][arg_name] = type_mapping[expected_type](arg_value)

return tool_call



{'location': {'type': 'str'}, 'unit': {'type': 'str'}}
location Charlotte
<class 'str'>
unit Fahrenheit
<class 'str'>
